In [0]:
from pyspark.sql.functions import *

In [0]:
df_silver = spark.read.table("nyc_taxi.nyc_schema.silver")
df_zone_lookup = spark.read.table("nyc_taxi.nyc_schema.zone_lookup")

In [0]:
df_joined = df_silver.join(
    broadcast(df_zone_lookup),
    df_silver.PULocationID == df_zone_lookup.location_id,
    how="left"
)

In [0]:
df_revenue_zone_hour = df_joined.groupBy(
    "pickup_date", "pickup_hour", "zone"
).agg(
    count("*").alias("trip_count"),
    round(sum("total_amount"), 2).alias("total_revenue"),
    round(avg("total_amount"), 2).alias("avg_revenue_per_trip"),
    round(avg("fare_per_mile"), 2).alias("avg_fare_per_mile")
).orderBy("pickup_date", "pickup_hour", "zone")

display(df_revenue_zone_hour)

In [0]:
df_revenue_zone_hour.write.mode("overwrite").saveAsTable("nyc_taxi.nyc_schema.revenue_zone_hour")

In [0]:
%sql
select count(*) from nyc_taxi.nyc_schema.revenue_zone_hour